###-----------------------

In [471]:
import numpy as np
import random
import turtle
from turtle import *
vn = turtle.Screen()
robo = turtle.Turtle()
robo.hideturtle()
robo.penup()
engel = turtle.Turtle()

#initialize
numpadim = [0,0]  #current numpad
hücrem = 0    #current cell
engelli = 0  #engel sayısı
blok = 1         # 
numpad = 10      # 9 hücreden oluşuyor, hücrelerin adları numpadle aynı,[0]kullanılmıyor
boy = 30         # robotun boyu ve eni 30cm olarak düşünülebilir
maxziyaret = 5  # aynı hücreye 5 kere giderse sonlanıyor
bitti = False    # True olunca sonlanıyor
currentyön = 0

yatay = 2  # yanyana olan numpad sayısı herbiri 90 cm olabilir (5 demek 4,5 metre)
dikey = yatay  # dikey olarak numpad sayısı  yatay x dikey

#######

#durum  -1 kullanılmıyor
#        0: gidilmedi
#        1: gidildifor j in range(yatay):
#        2: engelli (gidilemez)
ndurum = np.zeros((numpad),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
ndurum[0] = -1

#hücrelerin puanları
#engelliyse 0 
#her hücrenin başlangıç puanı 10
npuan = 10*np.ones((numpad),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
npuan[0] = -1

#her gidişte artan
nziyaret = np.zeros((numpad),dtype='int')

durum = np.zeros((yatay,dikey,numpad),dtype='int') 
puan = np.zeros((yatay,dikey,numpad),dtype='int') 
ziyaret = np.zeros((yatay,dikey,numpad),dtype='int') 



#Açılan pencerenin boyutlarını belirliyor
bigkoor = (3*boy) + 1
#sıfır,sıfır sol üst köşede
#0,0 left upper corner
mode('world')
setworldcoordinates(0,bigkoor,bigkoor,0) #Açılan pencerenin boyutlarını belirliyor
koor = 30

for x in range(yatay):
    for y in range(dikey):
        durum[x,y] = ndurum
        puan[x,y] = npuan 
        ziyaret[x,y] = nziyaret  

In [472]:
# değişken haline getirilecek
#cell numpad kurallarıyla
def cell(X,Y,number):
    return {
        1: (5+(koor*X),25 + (koor*Y)),
        2: (15+(koor*X),25 + (koor*Y)),
        3: (25+(koor*X),25 + (koor*Y)),
        4: (5+(koor*X),15 + (koor*Y)),
        5: (15+(koor*X),15 + (koor*Y)),
        6: (25+(koor*X),15 + (koor*Y)),
        7: (5+(koor*X),5 + (koor*Y)),
        8: (15+(koor*X),5 + (koor*Y)),
        9: (25+(koor*X),5 + (koor*Y))
    } [number]

    


#durduğu hücreye göre yönünü gideceği hücreye çeviriyor    
def yönbul(hücre):
    return {
        1: [ 0,270,180,90],
        2: [180,0,270,90],
        3: [180,270,0,90],
        4: [90,0,270,180],
        5: [90,180,0,270],
        6: [90,180,270,0],
        7: [90,0,180,270],
        8: [90,180,0,270],
        9: [90,180]
    } [hücre]


def komşubul(X,Y,current):
    return {
        1: [(X,Y,2),(X,Y,4),(X-1,Y,3),(X,Y+1,7)],
        2: [(X,Y,1),(X,Y,3),(X,Y,5),(X,Y+1,8)],
        3: [(X,Y,2),(X,Y,6),(X+1,Y,1),(X,Y+1,9)],
        4: [(X,Y,1),(X,Y,5),(X,Y,7),(X-1,Y,6)],
        5: [(X,Y,2),(X,Y,4),(X,Y,6),(X,Y,8)],
        6: [(X,Y,3),(X,Y,5),(X,Y,9),(X+1,Y,4)],
        7: [(X,Y,4),(X,Y,8),(X-1,Y,9),(X,Y-1,1)],
        8: [(X,Y,5),(X,Y,7),(X,Y,9),(X,Y-1,2)],
        9: [(X,Y,6),(X,Y,8),(X,Y-1,3),(X+1,Y,7)]
    } [current]
    
    return

def doğruyönedön(yer):
    X,Y,gidilecekYer = yer
    ind = komşubul(X,Y,hücrem)
    for index, item in enumerate(ind):
        if item==yer:
            dön(yönbul(hücrem)[index])
    return

In [473]:
#directions 
#0 - east 
#90 - south
#180 - west
#270 - north

def dön(yön):
    global currentyön
    
    turtle.setheading(yön)
    robo.setheading(yön)
    currentyön =yön
    return

In [474]:
#Alanı çiz
#Draw the board
robo.reset()

def çiz(x,y):
    robo.penup()
    robo.setpos(x,y)
    robo.pendown()
    robo.forward(boy)
    return


def numpadçiz(X,Y):

    
    dön(0)
    
    çiz(0+(koor*X),0  + (koor*Y))
    çiz(0+(koor*X),10 + (koor*Y))
    çiz(0+(koor*X),20 + (koor*Y))
    çiz(0+(koor*X),30 + (koor*Y))

    dön(270)

    çiz(30+(koor*X),30  + (koor*Y))
    çiz(20+(koor*X),30  + (koor*Y))
    çiz(10+(koor*X),30  + (koor*Y))
    çiz(0+(koor*X),30  + (koor*Y))
    return



In [475]:
def engelkoy(X,Y,sayı):
    global durum,puan,engelli
    
    #random engel üretirken üst üste aynı yere koyarsa bitti kontrolü hatalı oluyor
    if durum[X,Y,sayı]==0:
        engelli +=1
        engel.setpos(cell(X,Y,sayı))
        durum[X,Y,sayı]= 2
        puan[X,Y,sayı] = -1000  
        engel.stamp()

In [476]:
def Git(yer):
    global hücrem,durum,puan,ziyaret,X,Y,bitti
    if bitti == False:
        X,Y,gidilecekyer = yer
        doğruyönedön(yer)
        
        #Güncelleme
        hücrem = gidilecekyer
        ziyaret[yer] += 1
        durum[yer]=1
        puan[yer]-=0.5
        turtle.setpos(cell(X,Y,gidilecekyer))


In [477]:
def puanaGöreKomşuSeç():
    global puanlistesi,currentyön,komşulistesi
     
    puanlistesi = []
    komşulistesi = []
    yönlistesi = []
    
    #eksi X,Y oldumu liste dışı olacak   !!!!!!!!!!!!!!!!!!!1
    for yer in komşubul(X,Y,hücrem):
        if yer[0]>=0 and yer[1]>=0 and yer[0]<yatay and yer[1]<dikey:
            komşulistesi.append(yer)
        
    for yer in komşulistesi:
        puanlistesi.append(puan[yer])
        yönlistesi.append(yönbul(yer[2]))
        
    d = dict(zip(komşulistesi,puanlistesi))
    y = dict(zip(komşulistesi,yönlistesi))
    
    enYüksekPuanlıYer = sorted(d, key=d.__getitem__, reverse=True)[0]
    
    #-------gittiği yönde gitmeye devam etsin diye uğraşıyoruz----------
    ikinciYer = sorted(d, key=d.__getitem__, reverse=True)[1]
    if enYüksekPuanlıYer == ikinciYer:
        if  y[ikinciYer]==currentyön:
            print("düz gitti")
            if puan[ikinciYer]==-1000:
                bitti = True
            return ikinciYer
    #    ind = (ikinciYer, enYüksekPuanlıYer)
    #    for index, item in enumerate(ind):
    #        if yönbul(item[2]) == currentyön:
    #            return item
            
    #gidebileceği en iyi yer engelli ise sonlanıyor
    if puan[enYüksekPuanlıYer]==-1000:
        bitti = True
        
    return enYüksekPuanlıYer

In [478]:
def tara():
    global bitti
    while not bitti:
        Git(puanaGöreKomşuSeç())
        # bir hücreye 10 sefer gittiyse bitir
        if max(ziyaret.ravel()) >= maxziyaret :
            bitti = True
        # bütün gidilebilecek noktalara gittiyse bitir
        if sum(durum.ravel()) == (engelli)  + ((8)*yatay*dikey):
            bitti = True
    return

In [479]:
robo.hideturtle()
for x in range(yatay):
    for y in range(dikey):
        numpadçiz(x,y)


In [480]:
#engel şeklini oluşturuyor
engel.penup() #kalem yukarı çıkıyor: yani inene kadar çizmiyor

engel.color("yellow")
engel.shape("circle")
engel.shapesize(3,3)



In [481]:
#başlangıç
dön(90)
X=0
Y=0
hücrem = 7
durum[X,Y,hücrem] = 1
puan[X,Y,hücrem] -= 0.5
ziyaret[X,Y,hücrem] += 1
turtle.setpos(cell(0,0,7))
turtle.speed(1)

#engel konuyor
i=0
for i in range(7):
    a = random.sample(range(2), 1)
    b = random.sample(range(2), 1)
    c = random.sample(range(10), 1)
    x = a[0]
    y = b[0]
    no = c[0]
    engelkoy(x,y,no)
    
turtle.speed(1)    

In [482]:

turtle.penup()

turtle.color("pink")
turtle.shape("turtle")
turtle.speed(1)
turtle.shapesize(3,2)
#turtle.setpos(5,5)



In [483]:
#main
turtle.pendown()
tara()


In [484]:
puan

array([[[   -1,    10, -1000,     6, -1000,     7,     6,     7,     5,
             8],
        [   -1,     8, -1000,     7,     7,     8,     6, -1000, -1000,
             6]],

       [[   -1,     6,     6,     7,     7,     7,     7,     7,     7,
             8],
        [   -1,     6,     6,     6,     6,     6,     6,     6,     6,
             7]]])

# bug

### iyileştirme
